# Pokémon Name Generation with Keras

Generate new unique Pokémon names with a LSTM using Andrej Karpathy's famous [Char-RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) which he used to generate poetry. There are more information in the blog, but the concept is fairly simple. We want the build a next-character-in-text predictor. We will do this by using a window of fixed length as our input and the next char as output and then train a LSTM to perform this task. Since the network won't understand raw characters we need to encode each character to a character vectors with one-hot encoding.

In [1]:
import pandas as pd
import numpy as np
import keras
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import os

## Settings

In [2]:
step_length = 1    # The step length we take to get our samples from our corpus
epochs = 100       # Number of times we train on our full data
batch_size = 32    # Data samples in each training step
latent_dim = 64    # Size of our LSTM
dropout_rate = 0.2 # Regularization with dropout
model_path = os.path.realpath('./poke_gen_model.h5') # Location for the model
load_model = True # Enable loading model from disk
store_model = True # Store model to disk after training
verbosity = 1      # Print result for each epoch
gen_amount = 1500    # How many 

## Loading data

I have made a .txt where I have stored the names of Pokémon as rows. I have also done some ealy preprocessing like removing special characters and only using lowercase characters. To generate other things than Pokémon names the rows in this file can simply be replaced with some other text that one wishes to generate.

In [3]:
input_path = os.path.realpath('./input/names.txt')

input_names = []

print('Reading Pokénames from file:')
with open(input_path) as f:
    for name in f:
        name = name.rstrip()
        if len(input_names) < 10:
            print(name)
        input_names.append(name)
    print('...')

Reading Pokénames from file:
bulbasaur
chikorita
treecko
turtwig
victini
chespin
rowlet
ivysaur
bayleef
grovyle
...


## Preprocessing
- Concatenate all Pokémon names into a long string corpus.
- Build dicionaries to translate chars to indices in a binary char vector.
- Find a suitable sequence window, I base it on the longest name I find.

In [4]:
# Make it all to a long string
concat_names = '\n'.join(input_names).lower()

# Find all unique characters by using set()
chars = sorted(list(set(concat_names)))
num_chars = len(chars)

# Build translation dictionaries, 'a' -> 0, 0 -> 'a'
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

# Use longest name length as our sequence window
max_sequence_length = max([len(name) for name in input_names])

print('Total chars: {}'.format(num_chars))
print('Corpus length:', len(concat_names))
print('Number of names: ', len(input_names))
print('Longest name: ', max_sequence_length)

Total chars: 27
Corpus length: 6734
Number of names:  799
Longest name:  12


Make a training set where we take samples with sequence length as our input and the next char as label.

In [5]:
sequences = []
next_chars = []

# Loop over our data and extract pairs of sequances and next chars
for i in range(0, len(concat_names) - max_sequence_length, step_length):
    sequences.append(concat_names[i: i + max_sequence_length])
    next_chars.append(concat_names[i + max_sequence_length])

num_sequences = len(sequences)

print('Number of sequences:', num_sequences)
print('First 10 sequences and next chars:')
for i in range(10):
    print('X=[{}]   y=[{}]'.replace('\n', ' ').format(sequences[i], next_chars[i]).replace('\n', ' '))

Number of sequences: 6722
First 10 sequences and next chars:
X=[bulbasaur ch]   y=[i]
X=[ulbasaur chi]   y=[k]
X=[lbasaur chik]   y=[o]
X=[basaur chiko]   y=[r]
X=[asaur chikor]   y=[i]
X=[saur chikori]   y=[t]
X=[aur chikorit]   y=[a]
X=[ur chikorita]   y=[ ]
X=[r chikorita ]   y=[t]
X=[ chikorita t]   y=[r]


One-hot encoding our data into char vectors by using the translation dictionary from earlier.

#### Example

- 'a'   => [1, 0, 0, ..., 0]

- 'b'   => [0, 1, 0, ..., 0]

- 'c'   => [0, 0, 1, ..., 0]

- 'abc' => [[1, 0, 0, ..., 0], [0, 1, 0, ..., 0], [0, 0, 1, ..., 0]] 

In [6]:
X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=np.bool)
Y = np.zeros((num_sequences, num_chars), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for j, char in enumerate(sequence):
        X[i, j, char2idx[char]] = 1
    Y[i, char2idx[next_chars[i]]] = 1
    
print('X shape: {}'.format(X.shape))
print('Y shape: {}'.format(Y.shape))

X shape: (6722, 12, 27)
Y shape: (6722, 27)


/scratch/local/51977825/ipykernel_1962678/2355450511.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=np.bool)
/scratch/local/51977825/ipykernel_1962678/2355450511.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y = np.zeros((num_sequences, num_chars), dtype=np.bool)


## Build model

Build a standard LSTM network with: 

- Input shape: (max_sequence_length x num_chars) - representing our sequences.
- Output shape: num_chars - representing the next char coming after each sequence.
- Output activation: Softmax - since only one value should be 1 in output char vector.
- Loss: Categorical cross-entrophy - standard loss for multi-class classification.

In [7]:
model = Sequential()
model.add(LSTM(latent_dim, 
               input_shape=(max_sequence_length, num_chars),  
               recurrent_dropout=dropout_rate))
model.add(Dense(units=num_chars, activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                23552     
                                                                 
 dense (Dense)               (None, 27)                1755      
                                                                 
Total params: 25,307
Trainable params: 25,307
Non-trainable params: 0
_________________________________________________________________


2024-12-06 11:10:15.297575: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-06 11:10:16.157544: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78902 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:47:00.0, compute capability: 8.0
/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## Training

Watching the loss, doing cross-validation and all that good stuff is not that important here. The best model will not be found by optimizing some metric. We just want to strike a balance between a model that just output gibberish like 'sadsdaddddd' and model that memorizes the names it was trained on. For this it is better to just inspect the output and judge from that.

In [8]:
if load_model:
    model.load_weights(model_path)
else:
    
    start = time.time()
    print('Start training for {} epochs'.format(epochs))
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbosity)
    end = time.time()
    print('Finished training - time elapsed:', (end - start)/60, 'min')
    
if store_model:
    print('Storing model at:', model_path)
    model.save(model_path)

Storing model at: /blue/rcstudents/smaley/pokegan/name-generation/pokemon-name-generator/poke_gen_model.h5


## Generation

Generate names by starting with a real sequence from the corpus, continuously predicting the next char while updating the sequence. To get diversity the correct char is selected from a probability distribution based on the models prediction. This can also be furthered by something called temperature, which I didn't use here.

I also added some postprocessing to remove things I did not like manually. Some of this could possibly be done by teaking the network, but I was happy with the way the names looked overall. 

In [9]:
# Start sequence generation from end of the input sequence
sequence = concat_names[-(max_sequence_length - 1):] + '\n'

new_names = []

print('{} new names are being generated'.format(gen_amount))

while len(new_names) < gen_amount:
    
    # Vectorize sequence for prediction
    x = np.zeros((1, max_sequence_length, num_chars))
    for i, char in enumerate(sequence):
        x[0, i, char2idx[char]] = 1

    # Sample next char from predicted probabilities
    probs = model.predict(x, verbose=0)[0]
    probs /= probs.sum()
    next_idx = np.random.choice(len(probs), p=probs)   
    next_char = idx2char[next_idx]   
    sequence = sequence[1:] + next_char

    # New line means we have a new name
    if next_char == '\n':

        gen_name = [name for name in sequence.split('\n')][1]

        # Never start name with two identical chars, could probably also
        if len(gen_name) > 2 and gen_name[0] == gen_name[1]:
            gen_name = gen_name[1:]

        # Discard all names that are too short
        if len(gen_name) > 2:
            
            # Only allow new and unique names
            if gen_name not in input_names + new_names:
                new_names.append(gen_name.capitalize())

        if 0 == (len(new_names) % (gen_amount/ 10)):
            print('Generated {}'.format(len(new_names)))

1500 new names are being generated


2024-12-06 11:10:17.851964: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Generated 150
Generated 300
Generated 450
Generated 600
Generated 750
Generated 900
Generated 1050
Generated 1050
Generated 1050
Generated 1200
Generated 1350
Generated 1500


## Results

Here are the results. I personally cannot tell the difference between generated names and names of Pokémon I dont know. Sometimes there are giveaways, but overall the names are convincing and diverse!

In [10]:
print_first_n = min(10, gen_amount)

print('First {} generated names:'.format(print_first_n))
for name in new_names[:print_first_n]:
    print(name)

First 10 generated names:
Forytrus
Kraboys
Ectove
Panpow
Onfisk
Carchhoin
Hichel
Lugoan
Blastorde
Seedrold


Storing the results

In [11]:
concat_output = '\n'.join(sorted(new_names))
output_path = os.path.realpath('./output/generated_names.txt')

with open(output_path, 'w') as f:
    f.write(concat_output)